
<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
 <h2>Mentoría: Clasificación de Tumoresferas </h2>
</center>



<a name="exploratory_data_analysis"></a>
#### **Práctico de Análisis Exploratorio y Curación**
  


Importamos las librerías necesarias:

In [10]:
#! pip install SQLAlchemy

In [11]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine, text

## Agrupaciones:

### SQL

Vamos a explorar los datos por medio de algunas consultas (queries) al archivo original (con el que trabajaron en el TP1):

 * Cargar el archivo *'data/raw/fiji_datos_0al7mo_labels.csv'* a SQLite.
 * Realizar las siguientes consultas:

    1) Verificar la cantidad de datos cargados (les debería dar 1018).
  
    2) Listar las primeras 7 líneas con las columnas *'Area'*, *'Round'* con alias *'redondez'*, *Diameter* con alias *'diametro'*, *n_diam* con alias  *poblacion_celular* y *esferoide*.
     
     Realizar la misma consulta pero filtrando por los días 3 y 5.
  
    3) Consultar los distintos días y aparte las etiquetas presentes. Contar cuántas filas hay por día y luego consultar cuantas son *esferoide = 'si'*.   
    
     Probar con GROUP BY de dos columnas si pueden contabilizar 'si' y 'no' por día.
    
    4 ) En algún día en particular (a partir del 3er día), consultar cuántos datos:
        
    * Son esferoides "si" y "no".        

    * Tienen su diámetro entre 50 $\mu$m y 200$\mu$ m.




#### Usando SQL en Python:

In [12]:
## Para conectarse con sqlite:
engine = create_engine('sqlite:///fiji_datos.sqlite3', echo=True)

# Levantamos el archivo
url = 'https://raw.githubusercontent.com/luciabarg/datos_tumoresferas/main/data/raw/fiji_datos_0al7mo_labels.csv'
fiji_datos = pd.read_csv(url)

fiji_datos.to_sql('fiji_datos', con=engine, if_exists="replace")


2023-07-09 13:08:08,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:08:08,188 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fiji_datos")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("fiji_datos")


2023-07-09 13:08:08,191 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2023-07-09 13:08:08,458 INFO sqlalchemy.engine.Engine [generated in 0.07840s] [(0, 'Esferas_BT474_dia_0_well_1_100X_1_blob_1', 324.444, 1129.2718, 102.2051, 1129.2718, 102.2051, 67.3603, 1118.8859, 92.3913, 21.0598, 19.7011, 0.8985, 22.2117, 1648, 157, 23.4287, 19.7011, 1.0653, 0.9387, 0.95, 'si', 0, 20.9564, 1.491356796983188), (1, 'Esferas_BT474_dia_0_well_1_100X_1_blob_2', 497.5115, 1517.5528, 126.1022, 1517.5528, 126.1022, 82.8004, 1504.7554, 113.4511, 25.8152, 25.1359, 0.9119, 26.8751, 2215, 192, 16.1443, 25.1289, 1.0475, 0.9546, 0.9569, 'si', 0, 26.002, 2.8487332309604083), (2, 'Esferas_BT474_dia_0_well_1_100X_1_blob_3', 282.9078, 1314.2815, 126.0988, 1314.2815, 126.0988, 62.1586, 1304.3478, 116.8478, 19.7011, 18.3424, 0.9201, 20.437, 1920, 189, 15.4222, 18.3424, 1.0395, 0.962, 0.9519, 'si', 0, 19.3897, 1.1812580710195366), (3, 'Esferas_BT474_dia_0_well_1_100X_1_blob_5', 500.7421, 1189.2841, 212.6155, 1189.2841, 212.6155, 84.5571, 1175.9511, 200.4076, 26.4946, 25.1359, 0.8801, 26

INFO:sqlalchemy.engine.Engine:[generated in 0.07840s] [(0, 'Esferas_BT474_dia_0_well_1_100X_1_blob_1', 324.444, 1129.2718, 102.2051, 1129.2718, 102.2051, 67.3603, 1118.8859, 92.3913, 21.0598, 19.7011, 0.8985, 22.2117, 1648, 157, 23.4287, 19.7011, 1.0653, 0.9387, 0.95, 'si', 0, 20.9564, 1.491356796983188), (1, 'Esferas_BT474_dia_0_well_1_100X_1_blob_2', 497.5115, 1517.5528, 126.1022, 1517.5528, 126.1022, 82.8004, 1504.7554, 113.4511, 25.8152, 25.1359, 0.9119, 26.8751, 2215, 192, 16.1443, 25.1289, 1.0475, 0.9546, 0.9569, 'si', 0, 26.002, 2.8487332309604083), (2, 'Esferas_BT474_dia_0_well_1_100X_1_blob_3', 282.9078, 1314.2815, 126.0988, 1314.2815, 126.0988, 62.1586, 1304.3478, 116.8478, 19.7011, 18.3424, 0.9201, 20.437, 1920, 189, 15.4222, 18.3424, 1.0395, 0.962, 0.9519, 'si', 0, 19.3897, 1.1812580710195366), (3, 'Esferas_BT474_dia_0_well_1_100X_1_blob_5', 500.7421, 1189.2841, 212.6155, 1189.2841, 212.6155, 84.5571, 1175.9511, 200.4076, 26.4946, 25.1359, 0.8801, 26.6336, 1731, 309, 174.14

2023-07-09 13:08:08,472 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


1018

In [13]:
#Forma que les mostraron en la clase teórica:
query1 = "SELECT count(*) FROM fiji_datos;"

with engine.connect() as con:
    print(query1)
    rs = con.execute(text(query1))
    df_rs = pd.DataFrame(rs.fetchall())
    print(df_rs)

SELECT count(*) FROM fiji_datos;
2023-07-09 13:08:08,499 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:08:08,501 INFO sqlalchemy.engine.Engine SELECT count(*) FROM fiji_datos;


INFO:sqlalchemy.engine.Engine:SELECT count(*) FROM fiji_datos;


2023-07-09 13:08:08,503 INFO sqlalchemy.engine.Engine [generated in 0.00472s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00472s] ()


   count(*)
0      1018
2023-07-09 13:08:08,509 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [14]:
# Otra forma alternativa de usar sqlalchemy en Python

query1 = " SELECT count(*) FROM fiji_datos; "
with engine.connect() as con:
  query_result1 = pd.read_sql_query(text(query1), con)

query_result1

2023-07-09 13:08:08,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:08:08,520 INFO sqlalchemy.engine.Engine  SELECT count(*) FROM fiji_datos; 


INFO:sqlalchemy.engine.Engine: SELECT count(*) FROM fiji_datos; 


2023-07-09 13:08:08,522 INFO sqlalchemy.engine.Engine [generated in 0.00452s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00452s] ()


2023-07-09 13:08:08,526 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


,count(*)
0,1018


#### También pueden usar la versión [**gráfica online**](https://sqliteonline.com/) de SQLite ([SQLite instructivo](https://drive.google.com/drive/folders/1iW5OaNaNDafU4e4m87xID7HcEbvb1W0V?usp=drive_link) )


Extras que podrían ayudar:

* Documentación [*pandas.DataFrame.to_sql*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html) y [*pandas.read_sql_query*](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html).


* [Recursos SQL](https://drive.google.com/drive/folders/1EDSgMGbrRjNZX7m-GXunkQjNXUxY1NSn?usp=drive_link).
    
* Presentación en [MeTCamp](https://docs.google.com/presentation/d/1URSQt1sJ8Th8Y4J62zBv9x3I1lV0rckT/edit?usp=sharing&ouid=107018266094379471830&rtpof=true&sd=true).

* Súper súper buenas prácticas en [este repositorio](https://github.com/daianadte/wids-cba-2023).


### Resolvemos las queries usando SQLite.
1) Resuelta en la consigna.

2) Listar las primeras 7 líneas con las columnas 'Area', 'Round' con alias 'redondez', Diameter con alias 'diametro', n_diam con alias poblacion_celular y esferoide.



In [15]:
query2 = "SELECT Area, Round AS redondez, Diameter AS diametro, n_diam AS poblacion_celular, esferoide FROM fiji_datos LIMIT 7;"
with engine.connect() as con:
  query_result2 = pd.read_sql_query(text(query2), con)

query_result2

2023-07-09 13:08:08,543 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:08:08,551 INFO sqlalchemy.engine.Engine SELECT Area, Round AS redondez, Diameter AS diametro, n_diam AS poblacion_celular, esferoide FROM fiji_datos LIMIT 7;


INFO:sqlalchemy.engine.Engine:SELECT Area, Round AS redondez, Diameter AS diametro, n_diam AS poblacion_celular, esferoide FROM fiji_datos LIMIT 7;


2023-07-09 13:08:08,556 INFO sqlalchemy.engine.Engine [generated in 0.01298s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.01298s] ()


2023-07-09 13:08:08,559 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


,Area,redondez,diametro,poblacion_celular,Esferoide
0,324.4440,0.9387,20.95640,1.491357,si
1,497.5115,0.9546,26.00200,2.848733,si
2,282.9078,0.9620,19.38970,1.181258,si
3,500.7421,0.9730,25.88475,2.810370,si
4,492.8964,0.9817,25.33985,2.636596,si
5,406.1319,0.9533,23.27125,2.042164,si
6,333.6742,0.9496,20.80470,1.459204,si


Realizar la misma consulta pero filtrando por los días 3 y 5.

In [16]:
query3 = "SELECT Area, Round AS redondez, Diameter AS diametro, n_diam AS poblacion_celular, esferoide FROM fiji_datos WHERE dia IN (3, 5) LIMIT 7;"
with engine.connect() as con:
  query_result3 = pd.read_sql_query(text(query3), con)

query_result3

2023-07-09 13:08:08,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:08:08,590 INFO sqlalchemy.engine.Engine SELECT Area, Round AS redondez, Diameter AS diametro, n_diam AS poblacion_celular, esferoide FROM fiji_datos WHERE dia IN (3, 5) LIMIT 7;


INFO:sqlalchemy.engine.Engine:SELECT Area, Round AS redondez, Diameter AS diametro, n_diam AS poblacion_celular, esferoide FROM fiji_datos WHERE dia IN (3, 5) LIMIT 7;


2023-07-09 13:08:08,592 INFO sqlalchemy.engine.Engine [generated in 0.00709s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00709s] ()


2023-07-09 13:08:08,596 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


,Area,redondez,diametro,poblacion_celular,Esferoide
0,4635.9028,0.8392,78.82640,79.368212,si
1,5901.8342,0.9533,87.74420,109.467897,si
2,11259.0826,0.9699,122.97820,301.381032,si
3,22344.1746,0.7434,173.24380,842.567345,si
4,2700.3153,0.6096,63.88875,42.257587,no
5,5002.3445,0.8306,84.09790,96.380011,si
6,14646.5914,0.5600,164.88585,726.409779,no


3) Consultar los distintos días y aparte las etiquetas presentes. Contar cuántas filas hay por día y luego consultar cuantas son esferoide = 'si'.

Probar con GROUP BY de dos columnas si pueden contabilizar 'si' y 'no' por día.

In [17]:
# Consultamos las etiquetas presentes en la columna dia
query4 = "SELECT DISTINCT dia FROM fiji_datos;"
with engine.connect() as con:
  query_result4 = pd.read_sql_query(text(query4), con)

# Contamos cuántas filas hay por día
query5 = "SELECT dia, COUNT(*) AS total FROM fiji_datos GROUP BY dia;"
with engine.connect() as con:
  query_result5 = pd.read_sql_query(text(query5), con)

# Contamos cuántas filas hay por día para esferoides = 'si'
query6 = "SELECT dia, COUNT(*) AS total FROM fiji_datos WHERE esferoide = 'si' GROUP BY dia;"
with engine.connect() as con:
  query_result6 = pd.read_sql_query(text(query6), con)

# Hacemos un group by de esferoides y dia
query7 = "SELECT dia, esferoide, COUNT(*) AS total FROM fiji_datos GROUP BY dia, esferoide;"
with engine.connect() as con:
  query_result7 = pd.read_sql_query(text(query7), con)

print("\nQuery 4:")
print(query_result4)
print("\nQuery 5:")
print(query_result5)
print("\nQuery 6:")
print(query_result6)
print("\nQuery 7:")
print(query_result7)


2023-07-09 13:08:08,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:08:08,633 INFO sqlalchemy.engine.Engine SELECT DISTINCT dia FROM fiji_datos;


INFO:sqlalchemy.engine.Engine:SELECT DISTINCT dia FROM fiji_datos;


2023-07-09 13:08:08,635 INFO sqlalchemy.engine.Engine [generated in 0.01188s] ()


...

2023-07-09 13:08:08,667 INFO sqlalchemy.engine.Engine [generated in 0.00444s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00444s] ()


2023-07-09 13:08:08,672 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK



Query 4:
   dia
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7

Query 5:
   dia  total
0    0     94
1    1    342
2    2    155
3    3     43
4    4    254
5    5     52
6    6      4
7    7     74

Query 6:
   dia  total
0    0     74
1    1    253
2    2     70
3    3     18
4    4     48
5    5     28
6    6      2
7    7     29

Query 7:
    dia Esferoide  total
0     0        no     20
1     0        si     74
2     1        no     89
3     1        si    253
4     2        no     85
5     2        si     70
6     3        no     25
7     3        si     18
8     4        no    206
9     4        si     48
10    5        no     24
11    5        si     28
12    6        no      2
13    6        si      2
14    7        no     45
15    7        si     29


4) En algún día en particular (a partir del 3er día), consultar cuántos datos:

* Son esferoides "si" y "no".

* Tienen su diámetro entre 50 $\mu$m y 200$\mu$ m.

In [21]:
# Seleccionamos el dia 4 y contamos cuantos son esferoides si y no
query8 = "SELECT dia, esferoide, COUNT(*) AS total FROM fiji_datos WHERE dia = 4 GROUP BY esferoide;"
with engine.connect() as con:
  query_result8 = pd.read_sql_query(text(query8), con)

# Seleccionamos el dia 4 y filtramos en un rango para el diámetro
query9 = "SELECT dia, Diameter, COUNT(*) AS total FROM fiji_datos WHERE dia = 4 AND Diameter BETWEEN 50 AND 200;"
with engine.connect() as con:
  query_result9 = pd.read_sql_query(text(query9), con)

print("\nQuery 8:")
print(query_result8)

print("\nQuery 9:")
print(query_result9)


2023-07-09 13:23:11,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-07-09 13:23:11,093 INFO sqlalchemy.engine.Engine SELECT dia, esferoide, COUNT(*) AS total FROM fiji_datos WHERE dia = 4 GROUP BY esferoide;


INFO:sqlalchemy.engine.Engine:SELECT dia, esferoide, COUNT(*) AS total FROM fiji_datos WHERE dia = 4 GROUP BY esferoide;


2023-07-09 13:23:11,099 INFO sqlalchemy.engine.Engine [cached since 24.7s ago] ()


...

2023-07-09 13:23:11,115 INFO sqlalchemy.engine.Engine [cached since 24.69s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 24.69s ago] ()


2023-07-09 13:23:11,118 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK



Query 8:
   dia Esferoide  total
0    4        no    206
1    4        si     48

Query 9:
   dia  Diameter  total
0    4   92.6425     80


In [23]:
#cerramos la conexión
con.close()

### Python
Probar diferentes agrupaciones en Python.

In [24]:
#Por ej:
fiji_datos.groupby('dia').agg({'Esferoide': ['count']}).sort_values(by = 'dia')

,Esferoide
,count
dia,
0,94
1,342
2,155
3,43
4,254
5,52
6,4
7,74


Probamos distintos group by con python

In [45]:
# Agrupo por 'Esferoide' y contar las filas
fiji_datos.groupby('Esferoide').size().reset_index(name='total')

,Esferoide,total
0,no,496
1,si,522


In [43]:
# Agrupo por 'dia' y 'Esferoide' y cuento las filas
fiji_datos.groupby(['dia', 'Esferoide']).size().reset_index(name='total')

,dia,Esferoide,total
0,0,no,20
1,0,si,74
2,1,no,89
3,1,si,253
4,2,no,85
5,2,si,70
6,3,no,25
7,3,si,18
8,4,no,206
9,4,si,48


## Análisis y Curación de Datos

1) Para empezar a trabajar la tabla, primero hay que combinar los datasets, que fueron separados por día.

Estos son los siguientes archivos a unir:

*   "fiji_datos_0al7mo_modificado_dia_0.csv"
*   "fiji_datos_0al7mo_modificado_dia_1.csv"
*   "fiji_datos_0al7mo_modificado_dia_2.csv"
*   "fiji_datos_0al7mo_modificado_dia_3.csv"
*   "fiji_datos_0al7mo_modificado_dia_4.csv"
*   "fiji_datos_0al7mo_modificado_dia_5.csv"
*   "fiji_datos_0al7mo_modificado_dia_6.csv"
*   "fiji_datos_0al7mo_modificado_dia_7a.csv"
*   "fiji_datos_0al7mo_modificado_dia_7b.csv"

Se encuentran en la carpeta: **"data/02_EyC"** de la [carpeta compartida](https://drive.google.com/drive/folders/1RqGNySwACN33Qopmw0nHmj5Yv4M78ZXi?usp=drive_link) y en este repositorio.


Probar hacerlo con Python y/o SQL (por separado, para probar!). Verificar que llegan a la misma cantidad de filas y columnas.

2) Una vez armado el dataset modificado, explorar el dataset y buscar inconsistencias, por ej si hay valores nulos, duplicados, etc, tratando de encontrar todas las inconsistencias en los datos. Decidir el orden en cuál ir arreglándolas.

3) Corregir las inconsistencias que van encontrando y sobre los datos faltantes (¡verificar primero si los hay!), determinar cuál método sería el mas adecuado para imputarlos, teniendo en cuenta lo que aprendieron sobre los datos en el práctico de *Análisis y Visualización*.

4) Una vez que recuperan el dataset original, repasando lo que les dieron en la materia de *Exploración y Curación de datos* y ya pensando en las transformaciones que puedan servirles y que les conviene realizar para la clasificación en el práctico de aprendizaje supervisado:

  * ¿Sobre cuáles columnas realizarían encoding y de qué tipo?. Elegir un método e implementarlo.

  * Tenemos muchas columnas (¡aunque siempre puede haber mas!) por lo que es posible realizar algún método de reducción de dimensionalidad.

  * Escalar y/o normalizar? Esa es la cuestión. Teniendo en cuenta lo aprendido a partir de los datos, cuál método se ajustaría a estos datos?

  Sumar tales transformaciones como columnas para sumar características al conjunto de datos y guardar el achivo modificado.

Si se traban mucho mucho en la limpieza de los datos, este paso pueden hacerlo con el dataset original que usaron en el práctico 1.

5) Pueden identificar sesgos en la toma de los datos o en el etiquetado? Si les parece que es así, cómo sería un experimento o método para mejorarlos?  

6) Realizar una documentación técnica de los procesos realizados.






### OPCIONAL

7) Explorar la carpeta *py_folder*. El archivo *main.py* es solo un borrador. Agregar alguna función de transformación en el módulo de eda y que puedan ejecutarla corriendo el archivo.

 Para correr el archivo main, en una terminal:

    (.venv) $ python main.py

 [Pdoc](https://pdoc.dev/) es una elección común para la documentación en Python. Para usarlo en este proyecto (está un poco personalizado):

    1) Instalarlo:
       (.venv) $ pip install pdoc
    2) Crear la carpeta con la documentación
       (.venv) $ pdoc main.py utils/ -o ./documentacion/Docs -t ./documentacion/pdoc_templates/
